In [1]:
import tensorflow as tf

class CNNNet(tf.keras.Model):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=1)
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)
        self.conv2 = tf.keras.layers.Conv2D(filters=36, kernel_size=3, strides=1)
        self.pool2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(units=128)
        self.fc2 = tf.keras.layers.Dense(units=10)

    def call(self, x):
        x = self.pool1(tf.nn.relu(self.conv1(x)))
        x = self.pool2(tf.nn.relu(self.conv2(x)))
        x = self.flatten(x)
        x = tf.nn.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = CNNNet()

Metal device set to: Apple M2 Pro


2023-11-26 23:47:20.968186: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-26 23:47:20.968298: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
import tensorflow as tf
import numpy as np

def cust_conv2d(X, K):
    """实现卷积运算"""
    # 获取卷积核形状
    h, w = K.shape
    # 初始化输出值Y
    Y = np.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    # 实现卷积运算
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = np.sum(X[i:i + h, j:j + w] * K)
    return tf.convert_to_tensor(Y, dtype=tf.float32)

In [5]:
X = tf.constant([[1.0,1.0,1.0,0.0,0.0], [0.0,1.0,1.0,1.0,0.0],
                  [0.0,0.0,1.0,1.0,1.0],[0.0,0.0,1.0,1.0,0.0],[0.0,1.0,1.0,0.0,0.0]])
K = tf.constant([[1.0, 0.0,1.0], [0.0, 1.0,0.0],[1.0, 0.0,1.0]])
result = cust_conv2d(X, K)
print(result)

tf.Tensor(
[[4. 3. 4.]
 [2. 4. 3.]
 [2. 3. 4.]], shape=(3, 3), dtype=float32)


In [6]:
X = tf.constant([[10.,10.,10.,0.0,0.0,0.0], [10.,10.,10.,0.0,0.0,0.0], [10.,10.,10.,0.0,0.0,0.0],
                  [10.,10.,10.,0.0,0.0,0.0],[10.,10.,10.,0.0,0.0,0.0],[10.,10.,10.,0.0,0.0,0.0]])
Y = tf.constant([[0.0, 30.0,30.0,0.0], [0.0, 30.0,30.0,0.0],[0.0, 30.0,30.0,0.0],[0.0, 30.0,30.0,0.0]])

In [7]:
import tensorflow as tf

# 构造一个二维卷积层，它具有1个输出通道和形状为（3，3）的卷积核
conv2d = tf.keras.layers.Conv2D(1, kernel_size=(3, 3), use_bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、高度、宽度、通道），
# 其中批量大小和通道数都为1
X = tf.reshape(X, (1, 6, 6, 1))
Y = tf.reshape(Y, (1, 4, 4, 1))
lr = 0.001  # 学习率

# 定义损失函数
loss_fn = tf.keras.losses.MeanSquaredError()

for i in range(400):
    with tf.GradientTape() as tape:
        Y_pre = conv2d(X, training=True)
        loss = loss_fn(Y, Y_pre)
    
    gradients = tape.gradient(loss, conv2d.trainable_variables)
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    optimizer.apply_gradients(zip(gradients, conv2d.trainable_variables))
    
    if (i + 1) % 100 == 0:
        print(f'epoch {i+1}, loss {loss.numpy().sum():.4f}')

epoch 100, loss 0.0005
epoch 200, loss 0.0000
epoch 300, loss 0.0000
epoch 400, loss 0.0000


In [8]:
# 获取卷积层的权重列表
weights = conv2d.get_weights()

# 获取权重矩阵并调整形状
weight_matrix = tf.reshape(weights[0], (3, 3))

In [9]:
weight_matrix

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.68979245,  0.3885317 , -0.9557028 ],
       [ 1.3078194 , -0.316953  , -0.64847624],
       [ 1.0023887 , -0.07157984, -1.3958199 ]], dtype=float32)>

In [10]:
import tensorflow as tf
import numpy as np

def corr2d_mutil_in(X, K):
    """实现多输入通道的卷积运算"""
    h, w = K.shape[1], K.shape[2]
    # 初始化输出值
    value = np.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    # 实现多输入通道的卷积运算
    for x, k in zip(X, K):
        value = value + cust_conv2d(x, k)
    return tf.convert_to_tensor(value, dtype=tf.float32)